<a href="https://colab.research.google.com/github/adalves-ufabc/2024.Q2-PLN/blob/main/2024_Q2_PLN_AULA_13_Notebook_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2024-Q2]**
Prof. Alexandre Donizeti Alves

## **LangChain**
---



In [ ]:
#@title Instalando o pacote LangChain
!pip install langchain -q U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 950.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.0 MB/s eta 0:00:00


In [ ]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

0.2.12


In [ ]:
#@title Integração com o pacote da OpenAI

!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
#@title Definindo a chave da API da OpenAI

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## **Extração de Informação**

**Extração de Informação** (EI) é o processo de identificar e extrair dados estruturados a partir de texto não estruturado. O objetivo é transformar o texto bruto em informações organizadas que podem ser facilmente analisadas e usadas para outras tarefas, como a construção de bases de dados ou a realização de análises.

#### **Construir uma cadeia de extração**

***Schema***

Primeiro, precisamos descrever quais informações queremos extrair do texto.

Usaremos o `Pydantic` para definir um esquema de exemplo para extrair informações pessoais.

In [ ]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field

class Pessoa(BaseModel):
    """Informações sobre uma pessoa."""

    # ^ Doc-string para a entidade Pessoa.
    # Este doc-string é enviado ao LLM como a descrição do esquema Pessoa,
    # e pode ajudar a melhorar os resultados da extração.

    # Observe que:
    # 1. Cada campo é `opcional` -- isso permite que o modelo opte por não extraí-lo!
    # 2. Cada campo tem uma `descrição` -- essa descrição é usada pelo LLM.
    # Ter uma boa descrição pode ajudar a melhorar os resultados da extração.
    nome: Optional[str] = Field(default=None, description="O nome da pessoa")
    cor_cabelo: Optional[str] = Field(
        default=None, description="A cor do cabelo da pessoa, se conhecida"
    )
    altura_metros: Optional[str] = Field(
        default=None, description="Altura medida em metros"
    )

Aqui estão duas boas práticas ao definir um esquema:

- Documente os atributos e o próprio esquema: essas informações são enviadas ao LLM e usadas para melhorar a qualidade da extração de informações.
- Não force o LLM a inventar informações! Acima, usamos `Optional` para os atributos, permitindo que o LLM retorne `None` se não souber a resposta.

**Extrator**

Vamos criar um extrator de informações usando o esquema definido acima.

In [ ]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Defina um prompt personalizado para fornecer instruções e qualquer contexto adicional.
# 1) Você pode adicionar exemplos ao modelo de prompt para melhorar a qualidade da extração.
# 2) Introduza parâmetros adicionais para levar o contexto em consideração (por exemplo, incluir metadados
#    sobre o documento do qual o texto foi extraído.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um algoritmo de extração especializado. "
            "Extraia apenas as informações relevantes do texto. "
            "Se você não souber o valor de um atributo solicitado para extração, "
            "retorne nulo para o valor do atributo.",
        ),
        ("human", "{texto}"),
    ]
)

Precisamos usar um modelo que suporte chamadas de funções/`ferramentas`.

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | modelo.with_structured_output( schema = Pessoa )

Vamos testar isso:

In [ ]:
texto = "Alan Smith tem 183 centímetros de altura e cabelo loiro."

chain.invoke({"texto": texto})

Pessoa(nome='Alan Smith', cor_cabelo='loiro', altura_metros='1.83')

**Interessante**

>

Extração é Generativa 🤯
>
Os LLMs são modelos generativos, então eles podem fazer coisas bastante interessantes, como extrair corretamente a altura da pessoa em metros, mesmo que tenha sido fornecida em outra unidade!

**Múltiplas Entidades**

Na maioria dos casos, você deve estar extraindo uma lista de entidades em vez de uma única entidade. Isso pode ser facilmente alcançado usando o `Pydantic` ao aninhar modelos dentro de outros modelos.

In [ ]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field

class Pessoa(BaseModel):
    """Informações sobre uma pessoa."""

    # ^ Doc-string para a entidade Pessoa.
    # Este doc-string é enviado ao LLM como a descrição do esquema Pessoa,
    # e pode ajudar a melhorar os resultados da extração.

    # Observe que:
    # 1. Cada campo é `opcional` -- isso permite que o modelo opte por não extraí-lo!
    # 2. Cada campo tem uma `descrição` -- essa descrição é usada pelo LLM.
    # Ter uma boa descrição pode ajudar a melhorar os resultados da extração.
    nome: Optional[str] = Field(default=None, description="O nome da pessoa")
    cor_cabelo: Optional[str] = Field(
        default=None, description="A cor do cabelo da pessoa, se conhecida"
    )
    altura_metros: Optional[str] = Field(
        default=None, description="Altura medida em metros"
    )

class Dados(BaseModel):
    """Dados extraídos sobre pessoas."""

    # Cria um modelo para que possamos extrair várias entidades.
    pessoas: List[Pessoa]

In [ ]:
chain = prompt | modelo.with_structured_output( schema = Dados)

texto = "Meu nome é João, meu cabelo é preto e eu tenho 1,83 metros de altura. Ana tem a mesma cor de cabelo que eu."

chain.invoke({"texto": texto})

Dados(pessoas=[Pessoa(nome='João', cor_cabelo='preto', altura_metros='1,83'), Pessoa(nome='Ana', cor_cabelo='preto', altura_metros=None)])

#### **Como lidar com textos longos ao realizar a extração**

Quando trabalhar com arquivos, como PDFs, é provável que você encontre texto que excede a janela de contexto do seu modelo de linguagem. Para processar esse texto, considere estas estratégias:

1. **Mudar o LLM**: escolha um LLM diferente que suporte uma janela de contexto maior.
2. **Força Bruta**: divida o documento em partes e extraia o conteúdo de cada parte.
3. **RAG (*Retrieval-Augmented Generation*)**: divida o documento em partes, indexe as partes e extraia conteúdo apenas de um subconjunto de partes que pareçam "relevantes".

Lembre-se de que essas estratégias têm diferentes compromissos, e a melhor estratégia provavelmente depende da aplicação que você está projetando!

Aqui iremos demonstrar como implementar as estratégias 2 e 3.

In [ ]:
!pip install -qU langchain_community

Precisamos de alguns dados de exemplo! Vamos baixar um artigo sobre carros da **Wikipedia** e carregá-lo como um `Document` do LangChain.

In [ ]:
import re

import requests
from langchain_community.document_loaders import BSHTMLLoader

response = requests.get("https://en.wikipedia.org/wiki/Car")

with open("carros.html", "w", encoding="utf-8") as f:
    f.write(response.text)

loader = BSHTMLLoader("carros.html")
documento = loader.load()[0]
documento.page_content = re.sub("\n\n+", "\n", documento.page_content)

In [ ]:
print(len(documento.page_content))

79838


**Definir o esquema**

Usaremos o `Pydantic` para definir o esquema das informações que desejamos extrair. Neste caso, extrair uma lista de principais desenvolvimentos (por exemplo, eventos históricos importantes) que incluam um ano e uma descrição.

>
Observe que também incluímos uma chave de evidência e instruímos o modelo a fornecer, literalmente, as sentenças relevantes do texto do artigo. Isso nos permite comparar os resultados da extração com (a reconstrução do modelo de) texto do documento original.

In [ ]:
class Desenvolvimento(BaseModel):
    """Informações sobre um desenvolvimento na história dos carros."""

    ano: int = Field(
        ..., description="O ano em que houve um desenvolvimento histórico importante."
    )
    descricao: str = Field(
        ..., description="O que aconteceu neste ano? Qual foi o desenvolvimento?"
    )
    evidencia: str = Field(
        ...,
        description="Repita literalmente a(s) frase(s) da qual as informações de ano e descrição foram extraídas.",
    )


class ExtracaoDados(BaseModel):
    """Informações extraídas sobre desenvolvimentos chave na história dos carros."""

    desenvolvimentos: List[Desenvolvimento]


# Defina um prompt personalizado para fornecer instruções e qualquer contexto adicional.
# 1) Você pode adicionar exemplos no template do prompt para melhorar a qualidade da extração.
# 2) Introduza parâmetros adicionais para considerar o contexto (por exemplo, incluir metadados
#    sobre o documento do qual o texto foi extraído.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um especialista em identificar desenvolvimentos históricos chave em textos. "
            "Extraia apenas desenvolvimentos históricos importantes. Não extraia nada se não houver informações importantes no texto.",
        ),
        ("human", "{texto}"),
    ]
)

**Criar um extrator**

Vamos selecionar um LLM. Como estamos usando chamadas de `ferramentas`, precisaremos de um modelo que suporte esse recurso.

In [ ]:
from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
extrator = prompt | modelo.with_structured_output(
    schema = ExtracaoDados,
    include_raw = False,
)

**Abordagem de força bruta**



Divida os documentos em partes (*chunks*) de modo que cada parte se encaixe na janela de contexto dos LLMs.

In [ ]:
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(
    chunk_size=2000,
    chunk_overlap=20,
)

textos = text_splitter.split_text(documento.page_content)

* `chunk_size=2000`: define o tamanho máximo de cada parte (*chunk*) em tokens. Neste caso, cada parte terá no máximo 2000 tokens.

* `chunk_overlap=20`: define o número de tokens que se sobrepõem entre partes adjacentes. Uma sobreposição de 20 tokens significa que os 20 tokens finais de uma parte serão incluídos no início da próxima parte. Isso ajuda a garantir que o contexto não seja perdido entre partes consecutivas.

Use a funcionalidade de processamento em lote (`batch`) para executar a extração em paralelo em cada parte!

In [ ]:
# Limite apenas às primeiras 3 partes
# para que o código possa ser executado rapidamente
partes = textos[:3]

resposta = extrator.batch(
    [{"texto": texto} for texto in partes],
    {"max_concurrency": 5},  # número máximo de chamadas simultâneas
)

**Mesclar resultados**

Após extrair dados das partes, será necessário mesclar as extrações.

In [ ]:
desenvolvimentos = []

for extracao in resposta:
    desenvolvimentos.extend(extracao.desenvolvimentos)

desenvolvimentos[:10]

[Desenvolvimento(ano=1966, descricao='O Toyota Corolla começou a ser produzido.', evidencia='The Toyota Corolla, which has been in production since 1966, is the best-selling series.'),
 Desenvolvimento(ano=1769, descricao='O inventor francês Nicolas-Joseph Cugnot construiu o primeiro veículo rodoviário movido a vapor.', evidencia='The French inventor Nicolas-Joseph Cugnot built the first steam-powered road vehicle in 1769.'),
 Desenvolvimento(ano=1808, descricao='O inventor suíço François Isaac de Rivaz projetou e construiu o primeiro automóvel movido a combustão interna.', evidencia='the Swiss inventor François Isaac de Rivaz designed and constructed the first internal combustion-powered automobile in 1808.'),
 Desenvolvimento(ano=1886, descricao='Carl Benz patenteou seu Benz Patent-Motorwagen, considerado o primeiro carro moderno.', evidencia='the modern car—a practical, marketable automobile for everyday use—was invented in 1886, when the German inventor Carl Benz patented his Benz 

In [ ]:
for desenvolvimento in sorted(desenvolvimentos, key=lambda x: x.ano):
    print(f"{desenvolvimento.ano}: {desenvolvimento.descricao}")

1769: O inventor francês Nicolas-Joseph Cugnot construiu o primeiro veículo rodoviário movido a vapor.
1808: O inventor suíço François Isaac de Rivaz projetou e construiu o primeiro automóvel movido a combustão interna.
1879: Benz recebeu uma patente para seu primeiro motor, que foi projetado em 1878.
1881: O inventor francês Gustave Trouvé demonstrou um carro de três rodas movido a eletricidade.
1886: Carl Benz patenteou seu Benz Patent-Motorwagen, considerado o primeiro carro moderno.
1888: Bertha Benz undertook the first road trip by car to prove the road-worthiness of her husband's invention.
1890: Daimler and Maybach founded Daimler Motoren Gesellschaft (DMG) in Cannstatt.
1892: Daimler sold their first car under the brand name Daimler.
1892: Rudolf Diesel was granted a patent for a 'New Rational Combustion Engine'.
1893: The first running, petrol-driven American car was built and road-tested by the Duryea brothers.
1895: Selden was granted a US patent for a two-stroke car engine.

**Abordagem baseada em *RAG***

***RAG*** (*Retrieval-Augmented Generation* - Geração aumentada de recuperação)

Outra ideia simples é dividir o texto em partes, mas em vez de extrair informações de cada parte, concentre-se apenas nas partes mais relevantes.


**Atenção**

Pode ser difícil identificar quais partes são relevantes.

Por exemplo, no artigo sobre carros que estamos usando aqui, a maior parte do artigo contém informações sobre desenvolvimentos chave. Então, ao usar ***RAG***, provavelmente estaremos descartando muitas informações relevantes.

Sugerimos experimentar com seu caso de uso e determinar se essa abordagem funciona ou não.

Para implementar a abordagem baseada em ***RAG***:

1. Divida seu(s) documento(s) em partes e indexe-os (por exemplo, em um `vetorstore`);
2. Adicione um passo de recuperação antes do extrator, utilizando o `vetorstore`.

***RAG*** (*Retrieval-Augmented Generation*) é uma abordagem que combina recuperação de informações com geração de texto. A ideia principal é melhorar a capacidade de um modelo de linguagem ao integrar informações recuperadas de uma base de dados ou índice com a geração de texto.

Como funciona o ***RAG***:

   * **Recuperação**: quando um modelo precisa gerar uma resposta ou extrair informações, primeiro ele realiza uma busca em uma base de dados ou índice para recuperar partes relevantes do texto ou documentos que podem ajudar a responder a pergunta ou completar a tarefa. Esta base de dados é frequentemente um `vetorstore`, que armazena *embeddings* de texto para rápida recuperação.

   * **Geração**: após recuperar as informações relevantes, o modelo de linguagem utiliza essas informações como contexto adicional para gerar uma resposta mais informada e precisa. Isso combina o conhecimento do modelo com dados específicos recuperados, aumentando a relevância e precisão da saída gerada.

***FAISS*** (*Facebook AI Similarity Search*) é uma biblioteca desenvolvida pelo Facebook AI Research para realizar busca eficiente e similaridade de alta dimensão em grandes conjuntos de dados. É projetada para lidar com grandes volumes de dados e facilitar a recuperação de informações baseadas em similaridade.

***FAISS*** é especializado em encontrar vetores (representações numéricas de dados) similares a um vetor de consulta.

***Vectorstore*** é um termo usado para descrever um tipo de armazenamento especializado em vetores, frequentemente utilizado em sistemas de recuperação de informações e modelos de aprendizado de máquina. Ele é projetado para gerenciar e buscar vetores de alta dimensão de maneira eficiente.

In [ ]:
!pip install -qU FAISS

Aqui está um exemplo simples que utiliza o `vetorstore` `FAISS`.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

textos = text_splitter.split_text(documento.page_content)
vectorstore = FAISS.from_texts(textos, embedding = OpenAIEmbeddings())

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 1}
)  # extrai apenas do primeiro documento

Neste caso, o extrator ***RAG*** está analisando apenas o principal documento.

In [ ]:
rag_extrator = {
    "texto": retriever | (lambda docs: docs[0].page_content)
} | extrator

In [ ]:
resposta = rag_extrator.invoke("Principais desenvolvimentos associados aos carros")

In [ ]:
for desenvolvimento in resposta.desenvolvimentos:
    print(desenvolvimento)

ano=2006 descricao='Crescimento de dois dígitos na receita e no número de membros de serviços de compartilhamento de carros nos EUA.' evidencia='...alguns serviços de compartilhamento de carros tiveram crescimento de dois dígitos na receita e no número de membros entre 2006 e 2007.'
ano=2020 descricao='Produção de 56 milhões de carros em todo o mundo, uma queda em relação a 67 milhões no ano anterior.' evidencia='Em 2020, havia 56 milhões de carros fabricados em todo o mundo, down from 67 million the previous year.'
ano=2020 descricao='A China produziu 20 milhões de carros, liderando a produção mundial.' evidencia='A indústria automotiva na China produz de longe o maior número (20 milhões em 2020)...'
ano=2020 descricao='Cerca de um bilhão de carros estão nas estradas ao redor do mundo.' evidencia='Ao redor do mundo, há cerca de um bilhão de carros nas estradas.'
ano=2019 descricao='Aumento da popularidade de sistemas de compartilhamento de bicicletas em cidades europeias e nos EUA.' e

In [ ]:
for desenvolvimento in sorted(resposta.desenvolvimentos, key=lambda x: x.ano):
    print(f"{desenvolvimento.ano}: {desenvolvimento.descricao}")

2006: Crescimento de dois dígitos na receita e no número de membros de serviços de compartilhamento de carros nos EUA.
2019: Aumento da popularidade de sistemas de compartilhamento de bicicletas em cidades europeias e nos EUA.
2019: Estudo sobre os benefícios de introduzir bairros de baixo tráfego em Londres, mostrando que os benefícios superam os custos em 100 vezes nos primeiros 20 anos.
2020: Produção de 56 milhões de carros em todo o mundo, uma queda em relação a 67 milhões no ano anterior.
2020: A China produziu 20 milhões de carros, liderando a produção mundial.
2020: Cerca de um bilhão de carros estão nas estradas ao redor do mundo.


**IMPORTANTE:**

Métodos diferentes têm suas próprias vantagens e desvantagens relacionadas a custo, velocidade e precisão.

Fique atento a esses problemas:

- Dividir o conteúdo em partes significa que o LLM pode falhar em extrair informações se essas informações estiverem espalhadas por vários fragmentos.
- Um grande sobreposição entre os fragmentos pode fazer com que a mesma informação seja extraída duas vezes, então esteja preparado para desduplicar!
- LLMs podem inventar dados. Se você estiver procurando por um único fato em um texto longo e usar uma abordagem de força bruta, pode acabar obtendo mais dados inventados.

#### **Como usar apenas o *prompting* para realizar a extração**

Recursos de chamada de `ferramentas` não são necessários para gerar saídas estruturadas a partir de LLMs. LLMs que conseguem seguir bem as instruções do *prompt* podem ser encarregados de fornecer informações em um formato específico.
>
Essa abordagem depende de projetar bons *prompts* e, em seguida, fazer o *parser* da saída dos LLMs para garantir que eles extraíam informações de forma eficaz.
>
Para extrair dados sem recursos de chamada de `ferramentas`:
>
1. Instrua o LLM a gerar texto seguindo um formato esperado (por exemplo, `JSON` com um determinado esquema);
2. Use *parsers* de saída para estruturar a resposta do modelo em um objeto Python desejado.

Nem todos os modelos suportam `.with_structured_output()`, já que nem todos os modelos têm suporte para chamadas de `ferramentas` ou modo `JSON`. Para esses modelos, você precisará solicitar diretamente ao modelo que use um formato específico e usar um *parser* de saída para extrair a resposta estruturada da saída bruta do modelo.


**Usando `PydanticOutputParser`**

O exemplo a seguir usa o `PydanticOutputParser` embutido para fazer o *parser* da saída de um modelo de *chat* solicitado a corresponder ao esquema `Pydantic` fornecido. Note que estamos adicionando `format_instructions` diretamente ao *prompt* a partir de um método no *parser*:

In [ ]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class Pessoa(BaseModel):
    """Informação sobre uma pessoa."""

    nome: str = Field(..., description="O nome da pessoa.")
    altura_metros: float = Field(
        ..., description="A altura da pessoa expressa em metros."
    )

class Pessoas(BaseModel):
    """Informações de identificação sobre todas as pessoas em um texto."""

    pessoas: List[Pessoa]

# parser
parser = PydanticOutputParser(pydantic_object=Pessoas)

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Responda à consulta do usuário. Envolva a saída em tags `json`\n{format_instructions}",
        ),
        ("human", "{consulta}"),
    ]
).partial(format_instructions = parser.get_format_instructions())

Vamos dar uma olhada em quais informações serão enviadas ao modelo:

In [ ]:
consulta = "Ana tem 23 anos e mede 1,83 metros de altura."

print(prompt.invoke(consulta).to_string())

System: Responda à consulta do usuário. Envolva a saída em tags `json`
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Informa\u00e7\u00f5es de identifica\u00e7\u00e3o sobre todas as pessoas em um texto.", "properties": {"pessoas": {"title": "Pessoas", "type": "array", "items": {"$ref": "#/definitions/Pessoa"}}}, "required": ["pessoas"], "definitions": {"Pessoa": {"title": "Pessoa", "description": "Informa\u00e7\u00e3o sobre uma pessoa.", "type": "object", "properties": {"nome": {"title": "Nome", "description": "O nome da pessoa.", "type": "string"}, "altura_metros": {"titl

E agora, vamos invocá-lo:

In [ ]:
chain = prompt | modelo | parser

resposta = chain.invoke({"consulta": consulta})

In [ ]:
resposta

Pessoas(pessoas=[Pessoa(nome='Ana', altura_metros=1.83)])

In [ ]:
type(resposta)

__main__.Pessoas

In [ ]:
resposta.pessoas[0].nome

'Ana'

In [ ]:
for pessoa in resposta.pessoas:
    print(pessoa.nome)

Ana


***Parser* Personalizado**

Você também pode criar um *prompt* e um *parser* personalizados com a **Linguagem de Expressão LangChain** (`LCEL`), usando uma função simples para fazer o *parser* da saída do modelo:

In [ ]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class Pessoa(BaseModel):
    """Informação sobre uma pessoa."""

    nome: str = Field(..., description="O nome da pessoa.")
    altura_metros: float = Field(
        ..., description="A altura da pessoa expressa em metros."
    )

class Pessoas(BaseModel):
    """Informações de identificação sobre todas as pessoas em um texto."""

    pessoas: List[Pessoa]

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Responda à consulta do usuário. Saia com sua resposta como JSON que  "
            "corresponda ao esquema fornecido: json\n{schema}\n. "
            "Certifique-se de envolver a resposta em tags json e .",
        ),
        ("human", "{consulta}"),
    ]
).partial(schema=Pessoas.schema())


# parser personalizado
def extrair_json(mensagem: AIMessage) -> List[dict]:
    """Extraia o conteúdo JSON de uma string em que o JSON está embutido entre as tags ```json e ```.

    Parâmetros:
        texto (str): O texto contendo o conteúdo JSON.

    Retorna:
        lista: Uma lista de strings JSON extraídas.
    """
    texto = mensagem.content

    # padrão da expressão regular para corresponder a blocos JSON
    padrao = r"```json(.*?)```"

    # correspondências não sobrepostas do padrão na string
    matches = re.findall(padrao, texto, re.DOTALL)

    # retorne a lista de strings JSON correspondentes, removendo quaisquer espaços em branco no início ou no final
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Falha no parser: {mensagem}")

In [ ]:
texto = "Ana tem 23 anos e mede 1,83 metros de altura."

print(prompt.format_prompt(consulta = texto).to_string())

System: Responda à consulta do usuário. Saia com sua resposta como JSON que  corresponda ao esquema fornecido: json
{'title': 'Pessoas', 'description': 'Informações de identificação sobre todas as pessoas em um texto.', 'type': 'object', 'properties': {'pessoas': {'title': 'Pessoas', 'type': 'array', 'items': {'$ref': '#/definitions/Pessoa'}}}, 'required': ['pessoas'], 'definitions': {'Pessoa': {'title': 'Pessoa', 'description': 'Informação sobre uma pessoa.', 'type': 'object', 'properties': {'nome': {'title': 'Nome', 'description': 'O nome da pessoa.', 'type': 'string'}, 'altura_metros': {'title': 'Altura Metros', 'description': 'A altura da pessoa expressa em metros.', 'type': 'number'}}, 'required': ['nome', 'altura_metros']}}}
. Certifique-se de envolver a resposta em tags json e .
Human: Ana tem 23 anos e mede 1,83 metros de altura.


In [ ]:
chain = prompt | modelo | extrair_json

resposta = chain.invoke({"consulta": texto})

In [ ]:
resposta

[{'pessoas': [{'nome': 'Ana', 'altura_metros': 1.83}]}]

In [ ]:
type(resposta)

list

In [ ]:
resposta[0]['pessoas'][0]['nome']

'Ana'

#### **Referências:**

> https://python.langchain.com/v0.2/docs/tutorials/extraction/

> https://python.langchain.com/v0.2/docs/how_to/extraction_long_text/

> https://python.langchain.com/v0.2/docs/how_to/extraction_parse/